In [9]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extended Kalman Filter

> 

## 概要

### Schur 補行列の正定値性

区分行列 $\mathbf M$ を

$$\mathbf M=\begin{pmatrix}\mathbf A & \mathbf B \\ \mathbf C & \mathbf D  \end{pmatrix}$$

としたとき、区画 $\mathbf D$ に関する Schur 補行列とは

$$\mathbf M/\mathbf D=\mathbf A-\mathbf B\mathbf D^{-1}\mathbf C$$

で与えられる。

**定理**

対称な行列 $\mathbf M$ を

$$\mathbf M=\begin{pmatrix}\mathbf A & \mathbf B \\ \mathbf B^T & \mathbf C\end{pmatrix}$$

とすると、 $\mathbf A\succ 0$ であるとき、 

$$\text{If}\ \mathbf C\succ 0, \text{then}\ (\mathbf M\succ 0\iff \mathbf M/\mathbf C\succeq 0)$$

が成り立つ。


**定理**

$\mathbb E$ を任意の期待値： $\mathbb E_{\mathbf x,\mathbf y}, \mathbb E_{\mathbf x\mid\mathbf y}, \mathbb E_{\mathbf y\mid\mathbf x}$ であるとする。

$$
\begin{pmatrix} \mathbb E[\mathbf f(\mathbf x,\mathbf y)\mathbf f(\mathbf x,\mathbf y)^T] & \mathbb E[\mathbf f(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T] \\ \mathbb E[\mathbf f(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T]^T & \mathbb E[\mathbf g(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T] \end{pmatrix}\succeq 0
$$

**証明**

$\mathbf h(\mathbf x,\mathbf y)$ を

$$\mathbf h(\mathbf x,\mathbf y)=\begin{pmatrix}\mathbf f(\mathbf x,\mathbf y) \\ \mathbf g(\mathbf x,\mathbf y)\end{pmatrix}$$

とすると、任意のベクトル $\mathbf u$ に対して

$$
\begin{split}
&\phantom{=}\mathbf u^T\begin{pmatrix} \mathbb E[\mathbf f(\mathbf x,\mathbf y)\mathbf f(\mathbf x,\mathbf y)^T] & \mathbb E[\mathbf f(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T] \\ \mathbb E[\mathbf f(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T]^T & \mathbb E[\mathbf g(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T] \end{pmatrix}\mathbf u \\
&= \mathbf u^T\mathbb E[\mathbf h(\mathbf x,\mathbf y)\mathbf h(\mathbf x,\mathbf y)^T]\mathbf u \\
&= \mathbb E[(\mathbf h(\mathbf x,\mathbf y)\mathbf u)^2] \\
&\succeq 0
\end{split}
$$

**定理**

$\mathbb E[\mathbf g(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T]\succ 0$ のとき、

$$\mathbb E[\mathbf f(\mathbf x,\mathbf y)\mathbf f(\mathbf x,\mathbf y)^T]\succeq\mathbb E[\mathbf f(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T]\mathbb E[\mathbf g(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T]^{-1}\mathbb E[\mathbf f(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T]^T$$

が成り立つ。

**証明**

$$\mathbf M=\begin{pmatrix} \mathbb E[\mathbf f(\mathbf x,\mathbf y)\mathbf f(\mathbf x,\mathbf y)^T] & \mathbb E[\mathbf f(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T] \\ \mathbb E[\mathbf f(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T]^T & \mathbb E[\mathbf g(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T] \end{pmatrix}$$

とすると、 $\mathbf M\succeq 0$ であり、よって $\mathbf M/\mathbb E[\mathbf g(\mathbf x,\mathbf y)\mathbf g(\mathbf x,\mathbf y)^T]\succeq 0$ である。


### Fisher 情報行列

$\mathbf w$ を未知変数、 $y$ を観測変数とする。
Fisher 情報行列を

$$
\begin{split}
I(\mathbf w) &=-\mathbb E_{y\mid \mathbf w}\left[\frac{\partial^2}{\partial \mathbf w^2}\ln p(y\mid\mathbf w)\right] \\
&= -\int\left(\frac{\partial^2}{\partial \mathbf w^2}\ln p(y\mid\mathbf w)\right)p(y\mid\mathbf w)dy \\
&\in\mathbb R^{n\times n}
\end{split}
$$

とする。各要素は

$$
I_{ij}=-\mathbb E_{y\mid\mathbf w}\left[\frac{\partial^2}{\partial w_i\partial w_j}\ln p(y\mid\mathbf w)\right]
$$

となる。

### Fisher 情報行列は対称行列である

$$
\begin{split}
I(\mathbf w) &=-\mathbb E_{y\mid \mathbf w}\left[\frac{\partial^2}{\partial \mathbf w^2}\ln p(y\mid\mathbf w)\right] \\
&= -\int\left(\frac{\partial^2}{\partial \mathbf w^2}\ln p(y\mid\mathbf w)\right)p(y\mid\mathbf w)dy \\
&= -\int\left(\frac{\partial}{\partial\mathbf w}\left[\frac{1}{p(y\mid\mathbf w)}\frac{\partial}{\partial\mathbf w}p(y\mid\mathbf w)\right]\right)p(y\mid\mathbf w)dy \\
&= -\int\left(-\left[\frac{1}{p(y\mid\mathbf w)}\right]^2\left[\frac{\partial}{\partial\mathbf w}p(y\mid\mathbf w)\right]\left[\frac{\partial}{\partial\mathbf w}p(y\mid\mathbf w)\right]^T+\frac{1}{p(y\mid\mathbf w)}\frac{\partial^2}{\partial\mathbf w^2}p(y\mid\mathbf w)\right)p(y\mid\mathbf w)dy \\
&= \int\left(\frac{1}{p(y\mid\mathbf w)}\frac{\partial}{\partial \mathbf w}p(y\mid\mathbf w)\right)\left(\frac{1}{p(y\mid\mathbf w)}\frac{\partial}{\partial \mathbf w}p(y\mid\mathbf w)\right)^Tp(y\mid\mathbf w)dy-\int\frac{\partial^2}{\partial\mathbf w^2}p(y\mid\mathbf w)dy \\
&= \int\left(\frac{\partial}{\partial\mathbf w}\ln p(y\mid\mathbf w)\right)\left(\frac{\partial}{\partial\mathbf w}\ln p(y\mid\mathbf w)\right)^Tp(y\mid\mathbf w)dy-\underbrace{\frac{\partial^2}{\partial\mathbf w^2}\int p(y\mid\mathbf w)dy}_\mathbf O \\
&= \mathbb E_{y\mid\mathbf w}\left[\left(\frac{\partial}{\partial\mathbf w}\ln p(y\mid\mathbf w)\right)\left(\frac{\partial}{\partial\mathbf w}\ln p(y\mid\mathbf w)\right)^T\right]
\end{split}
$$

であるから、Fisher 情報行列は

$$
I(\mathbf w) =-\mathbb E_{y\mid \mathbf w}\left[\frac{\partial^2}{\partial \mathbf w^2}\ln p(y\mid\mathbf w)\right] = \mathbb E_{y\mid\mathbf w}\left[\left(\frac{\partial}{\partial\mathbf w}\ln p(y\mid\mathbf w)\right)\left(\frac{\partial}{\partial\mathbf w}\ln p(y\mid\mathbf w)\right)^T\right]
$$

と表せる。

### クラメル・ラオ不等式

**定理**

$I(\mathbf w)\succ 0$ のとき、

$$\mathbb E_{y\mid\mathbf w}\left[(\mathbf w-\mathbf f(y))(\mathbf w-\mathbf f(y))^T\right]\succeq I^{-1}(\mathbf w)$$

が成り立つ。ただし、 $\mathbf f(y)$ は不偏推定値とする。つまり、次式が成り立つとする。

$$\mathbb E_{y\mid\mathbf w}[\mathbf f(y)]=\mathbf w$$

**証明**

Schur 補行列の正定値性より、

$$
\begin{split}
&\phantom{=}\mathbb E_{y\mid\mathbf w}\left[(\mathbf w-\mathbf f(y))(\mathbf w-\mathbf f(y))^T\right] \\
&\succeq \mathbb E_{y\mid\mathbf w}\left[(\mathbf w-\mathbf f(y))\left(\frac{\partial}{\partial \mathbf w}\ln p(y\mid\mathbf w)\right)^T\right]I^{-1}(\mathbf w)\mathbb E_{y\mid\mathbf w}\left[(\mathbf w-\mathbf f(y))\left(\frac{\partial}{\partial \mathbf w}\ln p(y\mid\mathbf w)\right)^T\right]^T
\end{split}
$$

ここで、

$$
\begin{split}
\mathbb E_{y\mid\mathbf w}\left[(\mathbf w-\mathbf f(y))\left(\frac{\partial}{\partial \mathbf w}\ln p(y\mid\mathbf w)\right)^T\right] &= \int\left(\mathbf w-\mathbf f(y)\right)\left(\frac{1}{p(y\mid\mathbf w)}\frac{\partial}{\partial\mathbf w}p(y\mid\mathbf w)\right)^Tp(y\mid\mathbf w)dy \\
&= \int \left(\mathbf w - \mathbf f(y)\right)\left(\frac{\partial}{\partial\mathbf w}p(y\mid\mathbf w)\right)^Tdy \\
&= \mathbf w\left( \frac{\partial}{\partial \mathbf w} \int p(y\mid\mathbf w) dy \right)^T-\frac{\partial}{\partial \mathbf w^T}\int \mathbf f(y) p(y\mid\mathbf w)dy \\
&= \underbrace{\mathbf w\left(\frac{\partial}{\partial \mathbf w} 1\right)^T}_\mathbf O-\frac{\partial}{\partial\mathbf w^T}\mathbf w \\
&= -\mathbf I
\end{split}
$$

が成り立つ。 $\mathbf I$ は単位行列を表す。


### ベイズ情報行列

$\mathbf w$ を未知変数、 $y$ を観測変数とする。
ベイズ情報行列を

$$
\begin{split}
J &=-\mathbb E_{\mathbf w, y}\left[\frac{\partial^2}{\partial \mathbf w^2}\ln p(\mathbf w, y)\right] \\
&= -\int\int\left(\frac{\partial^2}{\partial \mathbf w^2}\ln p(\mathbf w, y)\right)p(\mathbf w, y)dyd\mathbf w \\
&\in\mathbb R^{n\times n}
\end{split}
$$

とする。各要素は

$$
J_{ij}=-\mathbb E_{\mathbf w,y}\left[\frac{\partial^2}{\partial w_i\partial w_j}\ln p(\mathbf w,y)\right]
$$

となる。

### ベイズ情報行列は対称行列である

$$
\begin{split}
J &=-\mathbb E_{\mathbf w,y}\left[\frac{\partial^2}{\partial \mathbf w^2}\ln p(\mathbf w,y)\right] \\
&= -\int\int\left(\frac{\partial^2}{\partial \mathbf w^2}\ln p(\mathbf w,y)\right)p(\mathbf w,y)d\mathbf wdy \\
&= -\int\int\left(\frac{\partial}{\partial\mathbf w}\left[\frac{1}{p(\mathbf w,y)}\frac{\partial}{\partial\mathbf w}p(\mathbf w,y)\right]\right)p(\mathbf w,y)d\mathbf wdy \\
&= -\int\int\left(-\left[\frac{1}{p(\mathbf w,y)}\right]^2\left[\frac{\partial}{\partial\mathbf w}p(\mathbf w,y)\right]\left[\frac{\partial}{\partial\mathbf w}p(\mathbf w,y)\right]^T+\frac{1}{p(\mathbf w,y)}\frac{\partial^2}{\partial\mathbf w^2}p(\mathbf w,y)\right)p(\mathbf w,y)d\mathbf wdy \\
&= \int\int\left(\frac{1}{p(\mathbf w,y)}\frac{\partial}{\partial \mathbf w}p(\mathbf w,y)\right)\left(\frac{1}{p(\mathbf w,y)}\frac{\partial}{\partial \mathbf w}p(\mathbf w,y)\right)^Tp(\mathbf w,y)d\mathbf wdy-\int\int\frac{\partial^2}{\partial\mathbf w^2}p(\mathbf w,y)d\mathbf wdy \\
&= \int\int\left(\frac{\partial}{\partial\mathbf w}\ln p(\mathbf w,y)\right)\left(\frac{\partial}{\partial\mathbf w}\ln p(\mathbf w,y)\right)^Tp(\mathbf w,y)d\mathbf wdy-\underbrace{\frac{\partial^2}{\partial\mathbf w^2}\int\int p(\mathbf w,y)d\mathbf wdy}_\mathbf O \\
&= \mathbb E_{\mathbf w,y}\left[\left(\frac{\partial}{\partial\mathbf w}\ln p(\mathbf w,y)\right)\left(\frac{\partial}{\partial\mathbf w}\ln p(\mathbf w,y)\right)^T\right]
\end{split}
$$

であり、

$$
\begin{split}
J &=-\mathbb E_{\mathbf w,y}\left[\frac{\partial^2}{\partial \mathbf w^2}\ln p(\mathbf w,y)\right] \\
&= -\int\int\left(\frac{\partial^2}{\partial \mathbf w^2}\ln p(\mathbf w\mid y)\right)p(\mathbf w,y)d\mathbf wdy -\underbrace{\int\int\left(\frac{\partial^2}{\partial \mathbf w^2}\ln p(y)\right)p(\mathbf w,y)d\mathbf wdy}_\mathbf O \\
&= \mathbb E_{\mathbf w,y}\left[\left(\frac{\partial}{\partial\mathbf w}\ln p(\mathbf w\mid y)\right)\left(\frac{\partial}{\partial\mathbf w}\ln p(\mathbf w\mid y)\right)^T\right]
\end{split}
$$

であるから、ベイズ情報行列は

$$
\begin{split}
J &=-\mathbb E_{\mathbf w,y}\left[\frac{\partial^2}{\partial \mathbf w^2}\ln p(\mathbf w,y)\right] \\
&= \mathbb E_{\mathbf w,y}\left[\left(\frac{\partial}{\partial\mathbf w}\ln p(\mathbf w,y)\right)\left(\frac{\partial}{\partial\mathbf w}\ln p(\mathbf w,y)\right)^T\right] \\
&= \mathbb E_{\mathbf w,y}\left[\left(\frac{\partial}{\partial\mathbf w}\ln p(\mathbf w\mid y)\right)\left(\frac{\partial}{\partial\mathbf w}\ln p(\mathbf w\mid y)\right)^T\right]
\end{split}
$$

と表せる。

### 事後クラメル・ラオ不等式 (Posterior Cramér-Rao inequality)

**定理**

$J\succ 0$ のとき、

$$\mathbb E_{\mathbf w,y}\left[(\mathbf w-\mathbf f(y))(\mathbf w-\mathbf f(y))^T\right]\succeq J^{-1}$$

が成り立つ。ただし、次式が成り立つとする。

$$\lim_{\mathbf w_i\to\pm\infty}\mathbb E_{y\mid\mathbf w}[\mathbf f(y)-\mathbf w]p(\mathbf w)=\lim_{\mathbf w_i\to\pm\infty}\int (\mathbf f(y)-\mathbf w)p(\mathbf w,y)dy=\mathbf 0,\ i=1,\ldots,n$$

**証明**

Schur 補行列の正定値性より、

$$
\begin{split}
&\phantom{=}\mathbb E_{\mathbf w,y}\left[(\mathbf w-\mathbf f(y))(\mathbf w-\mathbf f(y))^T\right] \\
&\succeq \mathbb E_{\mathbf w,y}\left[(\mathbf w-\mathbf f(y))\left(\frac{\partial}{\partial \mathbf w}\ln p(\mathbf w,y)\right)^T\right]J^{-1}\mathbb E_{\mathbf w,y}\left[(\mathbf w-\mathbf f(y))\left(\frac{\partial}{\partial \mathbf w}\ln p(\mathbf w,y)\right)^T\right]^T
\end{split}
$$

ここで、

$$
\begin{split}
\mathbb E_{\mathbf w,y}\left[(\mathbf w-\mathbf f(y))\left(\frac{\partial}{\partial \mathbf w}\ln p(\mathbf w,y)\right)^T\right] &= \int\int\left(\mathbf w-\mathbf f(y)\right)\left(\frac{1}{p(\mathbf w,y)}\frac{\partial}{\partial\mathbf w}p(\mathbf w,y)\right)^Tp(\mathbf w,y)d\mathbf wdy \\
&= \int\int \left(\mathbf w - \mathbf f(y)\right)\left(\frac{\partial}{\partial\mathbf w}p(\mathbf w,y)\right)^Td\mathbf wdy
\end{split}
$$

が成り立つ。$ij$ 要素に対して部分積分すると、

$$
\begin{split}
& \left\{\int\int \left(\mathbf w - \mathbf f(y)\right)\left(\frac{\partial}{\partial\mathbf w}p(\mathbf w,y)\right)^Td\mathbf wdy\right\}_{ij} \\
&= \int\int(w_i-f_i(y))\left(\frac{\partial}{\partial w_j}p(\mathbf w,y)\right)d\mathbf wdy \\
&= \int\int\left[(w_i-f_i(y))p(\mathbf w,y)\right]_{w_j=-\infty}^{w_j=\infty}d\mathbf w_kdy-\int\int\frac{\partial}{\partial w_j}(w_i-f_i(y))p(\mathbf w,y)d\mathbf wdy \\
&= \int\left[\lim_{w_j\to\infty}\int(w_i-f_i(y))p(\mathbf w,y)dy-\lim_{w_j\to-\infty}\int(w_i-f_i(y))p(\mathbf w,y)dy\right]d\mathbf w_k - I_{ij} \\
&= -I_{ij}
\end{split}
$$

となる。$\mathbf w_k$ は $k\neq j$ となる全ての $k$ を添字とする $w$ を表す。 $I_{ij}$ とは単位行列 $\mathbf I$ の $ij$ 要素を表す。

In [10]:
#| default_exp EKF

In [11]:
#| hide
#| export
import jax.numpy as jnp
import jax.lax as lax
from jaxtyping import Array, Float, Int
from KalmanPaper import simple as sp
from typing import Tuple

### Logistic sigmoid 関数の微分

$$\frac{d}{dx}\sigma(x)=\sigma(x)\{1-\sigma(x)\}$$

In [12]:
#| export
def dxlosi(
    x: Float[Array, ""] # $x$
) -> Float[Array, ""]: # $(d/dx)\sigma(x)$
  return sp.losi(x) * (1 - sp.losi(x))

### `Ptt`: 推定誤差共分散行列 $\mathbf P_{t/t}$

$$\sigma_t=\sigma(\hat{\mathbf w}_{t/t-1}^T\mathbf x_t)$$
$$\mathbf P_{t/t}=\mathbf P_{t/t-1}-\frac{\sigma_t(1-\sigma_t)}{1+\sigma_t(1-\sigma_t)\mathbf x_t^T\mathbf P_{t/t-1}\mathbf x_t}(\mathbf P_{t/t-1}\mathbf x_t)(\mathbf P_{t/t-1}\mathbf x_t)^T$$

In [13]:
#| export
def Ptt(
    Ptm: Float[Array, "N N"], # $\mathbf P_{t/t-1}$
    w: Float[Array, "N"],   # $\hat{\mathbf w}_{t/t-1}$
    x: Float[Array, "N"],   # $\mathbf x_t$
) -> Float[Array, "N N"]:   # $\mathbf P_{t/t}$
  dsigma = dxlosi(w @ x)
  Ptmx = Ptm @ x
  return Ptm - (dsigma / (1 + dsigma * (x @ Ptmx))) * jnp.outer(Ptmx, Ptmx)

### `wtt`: 濾波推定値 $\hat{\mathbf w}_{t/t}$

$$\hat{\mathbf w}_{t/t}=\hat{\mathbf w}_{t/t-1}+\frac{1}{1+\sigma_t(1-\sigma_t)\mathbf x_t^T\mathbf P_{t/t-1}\mathbf x_t}\mathbf P_{t/t-1}\mathbf x_t(y_t-\sigma_t)$$

In [14]:
#| export
def wtt(
    Ptm: Float[Array, "N N"], # $\mathbf P_{t/t-1}$
    w: Float[Array, "N"],   # $\hat{\mathbf w}_{t/t-1}$
    x: Float[Array, "N"],   # $\mathbf x_t$
    y: Float[Array, "N"],   # $y_t$
) -> Float[Array, "N"]: # $\hat{\mathbf w}_{t/t}$
  dsigma = dxlosi(w @ x)
  Ptmx = Ptm @ x
  return w + ((y - sp.losi(w @ x)) / (1 + dsigma * (x @ Ptmx))) * Ptmx

### `EKF`: Extended Kalman Filter

$$N,T,\{ \mathbf x_t \}_{t=0,\ldots,T-1}, \{ y_t \}_{t=0,\ldots,T-1}, \boldsymbol\Gamma, \hat{\mathbf w}_{0/-1}, \mathbf P_{0/-1} $$
$$\to\{\hat{\mathbf w}_{t/t}\}_{t=0,\ldots,T-1},\{\mathbf P_{t/t}\}_{t=0,\ldots,T-1}$$

In [15]:
#| export
def EKF(
    N: Int, # $N$
    T: Int, # $T$
    x: Float[Array, "{T} {N}"], # $\{ \mathbf x_t \}_{t=0,\ldots,T-1}$
    y: Float[Array, "{T} {N}"], # $\{ y_t \}_{t=0,\ldots,T-1}$
    G: Float[Array, "{N} {N}"], # $\boldsymbol\Gamma$
    w0: Float[Array, "{N}"], # $\hat{\mathbf w}_{0/-1}$
    P0: Float[Array, "{N} {N}"], # $\mathbf P_{0/-1}$
) -> Tuple[Float[Array, "{T} {N}"], Float[Array, "{T} {N} {N}"]]: # $\{\hat{\mathbf w}_{t/t}\}_{t=0,\ldots,T-1},\{\mathbf P_{t/t}\}_{t=0,\ldots,T-1}$
    def step(carry, inputs):
        Ptm, wtm = carry
        xt, yt = inputs
        Ptt_ = Ptt(Ptm, wtm, xt)
        wtt_ = wtt(Ptm, wtm, xt, yt)
        return (Ptt_ + G, wtt_), (wtt_, Ptt_)
    
    _, (W, P) = lax.scan(
        step,
        (P0, w0),
        (x, y),
        length=T
    )
    return W, P

In [16]:
#| hide
import nbdev; nbdev.nbdev_export()